In [6]:
#!pip install gensim
#!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 3.1 MB/s eta 0:00:00
  Obtaining dependency information for simpful from https://files.pythonhosted.org/packages/8d/93/8448d3f1aa9d2911b8cba2602aaa1af85eb31a26d28b7b737f1fa5b40c02/simpful-2.11.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20430 sha256=c3c20d4c2faceb8fb025ea5560faec61c9e23b63873e13f7733bdec3d03818b8
  Stored in directory: /Users/dzz_/Library/Caches/pip/wheels/2d/1b/42/88a19f6b3896c2230d5053832f208976cddf70625885201d06
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3513 sha256=9ff0d36517aa65c82d6fd5267ff67be0dd1c89394c5028eb3e2a261081b68d7a
  Stored in directory: /Users/dzz_/Library/Caches/pip/wheels/5b/86/8f/7bb7f6472e2c84de7addfc1a5cd7fd647f00d8fb640da9ea9a
Successfully built fst-pso miniful
ERROR: Could not find a version that satisfies the require

In [2]:
# Import dependencies
import gensim
import pandas as pd

This is the data set we are using: https://www.kaggle.com/datasets/zynicide/wine-reviews

In [3]:
# Read in data from csv file and create a dataframe
df = pd.read_csv("Resources/winemag-data-130k-v2.csv")
df.head()

,id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [11]:
# word2vec model:
# Preprocess Data to tokenize words:
descriptions = df.description.apply(gensim.utils.simple_preprocess)
descriptions

0         [aromas, include, tropical, fruit, broom, brim...
1         [this, is, ripe, and, fruity, wine, that, is, ...
2         [tart, and, snappy, the, flavors, of, lime, fl...
3         [pineapple, rind, lemon, pith, and, orange, bl...
4         [much, like, the, regular, bottling, from, thi...
                                ...                        
129966    [notes, of, honeysuckle, and, cantaloupe, swee...
129967    [citation, is, given, as, much, as, decade, of...
129968    [well, drained, gravel, soil, gives, this, win...
129969    [dry, style, of, pinot, gris, this, is, crisp,...
129970    [big, rich, and, off, dry, this, is, powered, ...
Name: description, Length: 129971, dtype: object

In [37]:
# Create the model
model = gensim.models.Word2Vec(window=3, min_count=2, workers=4)

In [38]:
# Build its vocabulary
model.build_vocab(descriptions, progress_per=100)

In [39]:
# Train the model
model.train(descriptions, total_examples=model.corpus_count, epochs= model.epochs)

(18385688, 25486870)

In [16]:
# Save the model to use later
model.save("Resources/descriptions.model")

In [40]:
# Test the model looking for similar words
model.wv.most_similar("dry")

[('dryness', 0.49381473660469055),
 ('concentrated', 0.46048474311828613),
 ('sweet', 0.45760372281074524),
 ('weighty', 0.4367315173149109),
 ('textured', 0.4301201403141022),
 ('lean', 0.42820876836776733),
 ('clean', 0.41724711656570435),
 ('refreshing', 0.4122775197029114),
 ('dryish', 0.41144445538520813),
 ('structured', 0.4093798100948334)]

In [41]:
# Test the model with similarity of different words
model.wv.similarity(w1="sweet",w2="fruity")

0.32179892

In [ ]:
# Clean and transform data
# Drop null values
# Drop unnecessary columns

In [ ]:
# Transform words into vectors using NLP
    # Tokenize descriptions to split sentences into word components
    # Drop stop words
    # Lemmitization
# Separate data in target and features variables
# Preprocess data
# Divide into train and test data

In [ ]:
# Input vector data into ML algorithm
# Apply K-Means Clustering
# Generate scatter plot of results

In [ ]:
# Input vector data into ML algorithm
# Apply 2nd Model (TBD)
# Generate scatter plot of results

In [ ]:
# Classify using Random Forest

In [ ]:
# Print accuracy score and confusion matrix